In [1]:
install.packages(c("tweenr","polyclip","ggforce"))

Warning message in install.packages(c("tweenr", "polyclip", "ggforce")):
“installation of package ‘tweenr’ had non-zero exit status”
Warning message in install.packages(c("tweenr", "polyclip", "ggforce")):
“installation of package ‘polyclip’ had non-zero exit status”
Warning message in install.packages(c("tweenr", "polyclip", "ggforce")):
“installation of package ‘ggforce’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

